## Create geometry outline of each windowed TIF

In [ ]:
from pathlib import Path

import rasterio
import geopandas as gpd
from shapely.geometry import box

In [ ]:
split_dir = Path("../data/split")
out_dir = Path("../data/train_tier_1")

In [ ]:
def create_overview(tif_dir, out_path):
    bbs = {}
    crs = None
    for tif in tif_dir.iterdir():
        with rasterio.open(tif) as rd:
            b = rd.bounds
            crs = rd.crs
        bbs[tif.stem.split("_")[-1]] = box(b.left, b.bottom, b.right, b.top)
    gdf = gpd.GeoDataFrame(index=bbs.keys(), geometry=list(bbs.values()))
    gdf.crs = crs
    gdf["idd"] = gdf.index
    gdf.to_file(out_path, driver="GPKG")
    print(f"Created for {tif_dir}")

In [ ]:
for area in split_dir.iterdir():
    for tif_dir in area.iterdir():
        out_path = out_dir / area.name / f"{tif_dir.name}-labels" / "windows.gpkg"
        create_overview(tif_dir, out_path)